In [1]:
import sys
sys.path.insert(0, '/workspaces/wiki3-kg-project')

import dspy
import json
from pathlib import Path

from ontological_engineer import (
    configure_lm,
    StatementExtractor,
    StatementQualityJudge,
)
from ontological_engineer.judges import statement_quality_metric
from ontological_engineer.training import (
    load_stage1_config,
    load_trainset,
    load_devset,
    load_fewshot_examples,
    save_optimized_extractor,
)

## 1. Load Configuration and Artifacts

Load the config and datasets saved by `stage1_statements.ipynb`.

In [2]:
# Load stage1 config (saved with CID provenance)
training_dir = Path("/workspaces/wiki3-kg-project/data/training")

config = load_stage1_config(training_dir)
print(f"Loaded config:")
print(f"  Model: {config['model']}")
print(f"  API base: {config['api_base']}")
print(f"  Temperature: {config['temperature']}")
print(f"  Num fewshot: {config['num_fewshot']}")
print(f"  Config CID: {config.get('cid', 'N/A')}")

Loaded config:
  Model: qwen/qwen3-coder-30b
  API base: http://host.docker.internal:1234/v1
  Temperature: 0.7
  Num fewshot: 3
  Config CID: bafkreicshdgzdtbuwtk7efcce2e7bcjq2dskfkwrn3xrq4pjkjgzubwhva


In [3]:
# Configure LM from saved config
lm = configure_lm(
    model=config['model'],
    api_base=config['api_base'],
    temperature=config['temperature'],
)
print(f"Configured LM: {lm}")

Configured LM: <dspy.clients.lm.LM object at 0xffffb05bd8a0>


In [4]:
# Load training and dev sets (with provenance)
trainset = load_trainset(training_dir)
devset = load_devset(training_dir)

print(f"Loaded trainset: {len(trainset)} examples")
print(f"Loaded devset: {len(devset)} examples")

Loaded trainset: 2742 examples
Loaded devset: 686 examples


In [5]:
# Load few-shot examples
fewshot_examples = load_fewshot_examples(training_dir)
print(f"Loaded {len(fewshot_examples)} few-shot examples")

Loaded 3 few-shot examples


In [6]:
# Load baseline results for comparison
baseline_path = training_dir / "baseline_results.json"
if baseline_path.exists():
    with open(baseline_path) as f:
        baseline_results = json.load(f)
    baseline_score = baseline_results['score']
    print(f"Baseline score: {baseline_score:.2f}")
    print(f"Baseline CID: {baseline_results.get('cid', 'N/A')}")
else:
    print("⚠️ No baseline results found. Run stage1_statements.ipynb first.")
    baseline_score = None

Baseline score: 92.36
Baseline CID: bafkreifcbdiqhxekomajwh4fhpzo3azyli5dydmfimdrdp3kbawnedvnwu


## 2. MLflow Setup

Configure MLflow for optimization tracking.

In [7]:
import mlflow

MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"

try:
    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    mlflow.set_experiment("wiki3-kg-stage1-optimization")
    
    # Enable autologging with full optimizer tracking
    # Reference: https://dspy.ai/tutorials/optimizer_tracking/
    mlflow.dspy.autolog(
        log_compiles=True,
        log_evals=True,
        log_traces_from_compile=True
    )
    
    print(f"✅ MLflow configured")
    print(f"   Tracking URI: {MLFLOW_TRACKING_URI}")
    print(f"   Experiment: wiki3-kg-stage1-optimization")
    MLFLOW_ENABLED = True
except Exception as e:
    print(f"⚠️ MLflow not available: {e}")
    MLFLOW_ENABLED = False

2025/12/20 18:46:04 INFO mlflow.tracking.fluent: Experiment with name 'wiki3-kg-stage1-optimization' does not exist. Creating a new experiment.


✅ MLflow configured
   Tracking URI: http://127.0.0.1:5000
   Experiment: wiki3-kg-stage1-optimization


## 3. MIPROv2 Optimization

Run prompt optimization using MIPROv2.

In [8]:
from dspy.teleprompt import MIPROv2

# Configure optimizer
optimizer = MIPROv2(
    metric=statement_quality_metric,
    auto="light",  # "light", "medium", or "heavy"
)

NUM_FEWSHOT = config['num_fewshot']

print(f"Optimizing with {len(trainset)} training examples...")
print(f"Using {NUM_FEWSHOT} few-shot demos for bootstrapping...")
print(f"MIPROv2 mode: auto='light'")

Optimizing with 2742 training examples...
Using 3 few-shot demos for bootstrapping...
MIPROv2 mode: auto='light'


In [9]:
# Run optimization
optimized_extractor = optimizer.compile(
    StatementExtractor(),
    trainset=trainset,
    max_bootstrapped_demos=NUM_FEWSHOT,
)

print("\n✅ Optimization complete!")
if MLFLOW_ENABLED:
    print(f"📊 View traces in MLflow UI: {MLFLOW_TRACKING_URI}")

2025/12/20 18:46:12 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '8f5c4c5bf86e4340a656f181d328fc80', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current dspy workflow
2025/12/20 18:46:12 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 10
minibatch: True
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 100

2025/12/20 18:46:12 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/12/20 18:46:12 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/12/20 18:46:12 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


  0%|          | 3/1742 [00:36<5:48:18, 12.02s/it]

Bootstrapped 3 full traces after 3 examples for up to 1 rounds, amounting to 3 attempts.


Bootstrapping set 4/6


...

2025/12/20 20:32:56 INFO dspy.evaluate.evaluate: Average Metric: 95.81999999999991 / 100 (95.8%)


2025/12/20 20:32:57 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 95.82
2025/12/20 20:32:57 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [92.7, 95.0, 95.82]
2025/12/20 20:32:57 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 95.82
2025/12/20 20:32:57 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/12/20 20:32:57 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/12/20 20:32:57 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 95.82!


🏃 View run eval_full_2 at: http://127.0.0.1:5000/#/experiments/2/runs/0b9f15028333461ba088e9f0d42ec3fa
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


🏃 View run hilarious-roo-68 at: http://127.0.0.1:5000/#/experiments/2/runs/8f5c4c5bf86e4340a656f181d328fc80
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2

✅ Optimization complete!
📊 View traces in MLflow UI: http://127.0.0.1:5000


[Trace(trace_id=tr-ae9aeb5bb1a0c3cbb7e2f1a412c91806), Trace(trace_id=tr-5bcb80e848a3bc27751bc383e3497908), Trace(trace_id=tr-f0393c7446c24ed59045f045cc4ac867), Trace(trace_id=tr-f07fc99551559bf06b61a192c5620ff5), Trace(trace_id=tr-b5ce33b472a45adec1ac77e7acd1be94), Trace(trace_id=tr-ffc87ea980918f45b9561058c57cf207), Trace(trace_id=tr-b2d7d4d243d04e7de345a2ee5401d361), Trace(trace_id=tr-039b45ba8744b2d6c03d8befd4b28c7a), Trace(trace_id=tr-2e65362ca6598de62644ff1a4d2c3b72), Trace(trace_id=tr-84ecc9ae3c0bd9341b43e2249fc39171)]

## 4. Evaluate Optimized Extractor

In [ ]:
# Evaluate on dev set
EVAL_SIZE = len(devset)

evaluator = dspy.Evaluate(
    devset=devset[:EVAL_SIZE],
    metric=statement_quality_metric,
    num_threads=1,
    display_progress=True,
)

optimized_result = evaluator(optimized_extractor)
optimized_score = optimized_result.score if hasattr(optimized_result, 'score') else float(optimized_result)

print(f"\nOptimized score: {optimized_score:.2f}")
if baseline_score is not None:
    improvement = optimized_score - baseline_score
    print(f"Baseline score:  {baseline_score:.2f}")
    print(f"Improvement:     {improvement:+.2f}")

## 5. Inspect Optimized Prompts

In [ ]:
print("Optimized extractor configuration:")
print("=" * 60)

if hasattr(optimized_extractor, 'demos'):
    print(f"\nDemonstrations: {len(optimized_extractor.demos)}")
    for i, demo in enumerate(optimized_extractor.demos[:2], 1):
        print(f"  Demo {i}: {demo.section_context[:50]}...")

if hasattr(optimized_extractor, 'signature'):
    print(f"\nSignature: {optimized_extractor.signature}")

## 6. Save Optimized Model

Save with CID provenance for reproducibility.

In [ ]:
# Save optimized extractor with provenance
output_path = save_optimized_extractor(
    extractor=optimized_extractor,
    output_dir=training_dir,
    config=config,
    baseline_score=baseline_score,
    optimized_score=optimized_score,
)

print(f"✅ Saved optimized extractor to: {output_path}")

In [ ]:
# Save optimization results summary
results = {
    "baseline_score": baseline_score,
    "optimized_score": optimized_score,
    "improvement": optimized_score - baseline_score if baseline_score else None,
    "train_size": len(trainset),
    "eval_size": EVAL_SIZE,
    "num_fewshot": NUM_FEWSHOT,
    "optimizer": "MIPROv2",
    "auto_mode": "light",
    "config_cid": config.get('cid'),
}

results_path = training_dir / "optimization_results.json"
with open(results_path, "w") as f:
    json.dump(results, f, indent=2)

print(f"Saved optimization results to: {results_path}")
print(json.dumps(results, indent=2))

## Summary

This notebook:
1. Loaded config and datasets from `stage1_statements.ipynb`
2. Ran MIPROv2 prompt optimization
3. Evaluated the optimized extractor
4. Saved the optimized model with provenance

**Next steps**:
- Review optimization traces in MLflow UI
- Run Stage 2: Schema matching
- Run Stage 3: RDF generation